# install dependecnies

In [2]:
!pip install -q datasets
!pip install -q transformers
!pip install -q umap-learn
!pip install -q bertviz
!pip install -q accelerate
!pip install -q seqeval
!pip install -q tqdm
!pip install -q scikit-learn 
!pip install -q transformers
!pip install -q huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new 

In [39]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.


In [42]:
!git lfs install

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log


In [36]:
from huggingface_hub import notebook_login
notebook_login()

# Imports

In [ ]:
import os
import requests
import tqdm
import json
import pandas as pd
import numpy as np
import zipfile
from collections import Counter

# Hyperparameters

In [ ]:
file_id = '1w3IQMK3PmXH-Bq6Lt_P8wxGPvr5olrZT'
destination = 'pfam.zip'
hf_name = "t5-small"
n_families_of_interest = 1000
data_dirpath = "pfam"
aminoacid_separate_by = ""
max_length = 512
tokenizer_folder = "PROTNAME_tok"
save_folder = "PROTNAME"
vocab_size=32

# Utils

In [ ]:
def format_size(value):
  if value >= 1024**3:
    return f"{round(value / 1024**3 , 3)} GB"
  elif value >= 1024**2:
    return f"{round(value / 1024**2 , 3)} MB"
  elif value >= 1024:
    return f"{round(value / 1024 , 3)} KB"
  return f"{value} B"

In [ ]:
# Code taken from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    params = { 'id' : id, 'confirm' : 1 }
    response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    
    pb = tqdm.tqdm(response.iter_content(CHUNK_SIZE))
    b_total = 0
    with open(destination, "wb") as f:
        for i,chunk in enumerate(pb):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                b_total += len(chunk)

                if i % 1000 == 0:
                  pb.set_description(f"written : {format_size(b_total)}")
                i += 1
    print("\n")
    print(f"saved all the data to {destination}. total size : {format_size(os.stat(destination).st_size)}")

# Get the dataset

In [ ]:
download_file_from_google_drive(file_id, destination)
with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall(".")
!mv random_split pfam

written : 468.781 MB: : 15780it [00:10, 1503.09it/s]




saved all the data to pfam.zip. total size : 493.095 MB


# Read data

In [ ]:
def read_all_shards(partition='dev', data_dir = data_dirpath):
    shards = []
    for fn in os.listdir(os.path.join(data_dir, partition)):
        with open(os.path.join(data_dir, partition, fn)) as f:
            shards.append(pd.read_csv(f, index_col=None))
    
    return pd.concat(shards)

def read_all_data_initial():
  global train, test, dev, all_train_ds_size, all_test_ds_size, all_dev_ds_size

  test = read_all_shards('test')
  dev = read_all_shards('dev')
  train = read_all_shards('train')

  partitions = {'test': test, 'dev': dev, 'train': train}
  for name, df in partitions.items():
      print('Dataset partition "%s" has %d sequences' % (name, len(df)))

  all_train_ds_size = len(train)
  all_test_ds_size = len(test)
  all_dev_ds_size = len(dev)

  train.reset_index(inplace=True, drop=True)
  dev.reset_index(inplace=True, drop=True)
  test.reset_index(inplace=True, drop=True)

def get_cumulative(data):
    counter = Counter(data['family_accession'])
    print(f"how many labels : {len(counter)}")
    
    datasetSize = len(data)
    xs = []
    x_labels = []
    ys = []

    t = 0
    cumulative = []

    for i,(x, y) in  enumerate(counter.most_common()):
        xs.append(i)
        x_labels.append(x)
        ys.append(y)
        t += y / datasetSize
        cumulative.append(t)
    return cumulative


# EXECUTION CODE
print('Available dataset partitions: ', os.listdir(data_dirpath))
read_all_data_initial()
cumulative = get_cumulative(train)
print(f"{n_families_of_interest} classes is {100 * round( cumulative[n_families_of_interest-1],3)} portion of training data")

familiesOfInterest = train.family_accession.value_counts()[:n_families_of_interest]

mask = train.family_accession.isin(familiesOfInterest.index.values)
train = train.loc[mask,:]

mask = dev.family_accession.isin(familiesOfInterest.index.values)
dev = dev.loc[mask,:]

mask = test.family_accession.isin(familiesOfInterest.index.values)
test = test.loc[mask,:]


################################################################################
train_seq = train['sequence']
dev_seq = dev['sequence']
test_seq = test['sequence']

################################################################################
train_sentences = train_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))
validation_sentences = dev_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))
test_sentences = test_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))

################################################################################
train_labels = train['family_accession'].apply(lambda x: x.split('.')[0])
validation_labels = dev['family_accession'].apply(lambda x: x.split('.')[0])
test_labels = test['family_accession'].apply(lambda x: x.split('.')[0])

Available dataset partitions:  ['train', 'test', 'random_split', 'dev']
Dataset partition "test" has 126171 sequences
Dataset partition "dev" has 126171 sequences
Dataset partition "train" has 1086741 sequences
how many labels : 17929
1000 classes is 40.400000000000006 portion of training data


In [ ]:
train_sentences

0          RIGIMTSGGDAPGMNLAIRAVARKALSSGLEAYGINYGFAGLVAGD...
4          SLWNILKKNIGKDLSKVAMPVTLNEPLNALQLLCEELEYSELLDDA...
7          FFGDVVGKPGRAAVLEHLPELRVRLRLDFVVVNGENAAGGFGLTRQ...
9          SDVRDMTPDQLQDELLKLKKTQFNLRFQGASGQLEKVHQMRQVRRD...
10                                      SLKFLNFAQNEFNGSIPESV
                                 ...                        
1086720    VLDIGCGNGKITAQIAERVPAGAVLGIDSSEQMISFASSHFAPADW...
1086724    LSVGLGCNRGTPAKAFEVAVTELFYDSSLNLQSIRNFASIDLKSDE...
1086729    QDWVASVSEPWLMLTPANGVGSTTGTVKIDTTLMSGRRTTDVAFLG...
1086735    DRIVAMSTISPIIAVVLALWSLEMRRGIYLDIALVLALLDFIMVVA...
1086738    ERLMPAFTGLSGSGIAYVLSFLHGMALAGTMAGIPYTESLRIVEAL...
Name: sequence, Length: 439493, dtype: object

In [ ]:
labels = set()

for v in tqdm.tqdm(train_labels):
  labels.add(v)

label_to_id = {k:v for v,k in enumerate(labels)}

print()
print(len(labels))

100%|██████████| 439493/439493 [00:00<00:00, 2166712.21it/s]


1000


In [ ]:
train_labels_as_int = train_labels.map(label_to_id)
val_labels_as_int = validation_labels.map(label_to_id)

In [ ]:
len(set(train_labels_as_int.values))

1000

In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict({"sentence": train_sentences, "labels": train_labels_as_int})
dataset_val = Dataset.from_dict({"sentence": validation_sentences, "labels": val_labels_as_int})

In [ ]:
dataset

Dataset({
    features: ['sentence', 'labels'],
    num_rows: 439493
})

In [ ]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training

tokenizer.train_from_iterator(iter(train_sentences), vocab_size=vocab_size, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=max_length)

In [ ]:
!mkdir {tokenizer_folder}
tokenizer.save_model(tokenizer_folder)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['PROTNAME_tok/vocab.json', 'PROTNAME_tok/merges.txt']

In [ ]:
datapoint = train_sentences.iloc[0]
encoded = tokenizer.encode(datapoint) 
print(f"datapoint : {datapoint}, len({len(datapoint)})")
print(encoded)

datapoint : RIGIMTSGGDAPGMNLAIRAVARKALSSGLEAYGINYGFAGLVAGDIHEFKAADLDDMVSQGGTMLYSARYPEFAQEESQLKGIEQLKKFGIDALVVIGGDGSYHGALRLTEHGYNTIGLPGTIDNDIPFTDFTIGFDTALNTAVDAIDKIRDTAKSHQRVFAVQVMGRNAADIALWAGVASGADAVIAPGFDYDVEAIANKLKKNRANGKDYGIIVIAEGDANSDAAPEFIDQLKQYGDFDARATVIGHVQRGGVPSAKDRVLASKMGAYAVELL, len(276)
Encoding(num_tokens=278, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [ ]:
print(encoded.ids)

[0, 54, 45, 43, 45, 49, 56, 55, 43, 43, 40, 37, 52, 43, 49, 50, 48, 37, 45, 54, 37, 58, 37, 54, 47, 37, 48, 55, 55, 43, 48, 41, 37, 61, 43, 45, 50, 61, 43, 42, 37, 43, 48, 58, 37, 43, 40, 45, 44, 41, 42, 47, 37, 37, 40, 48, 40, 40, 49, 58, 55, 53, 43, 43, 56, 49, 48, 61, 55, 37, 54, 61, 52, 41, 42, 37, 53, 41, 41, 55, 53, 48, 47, 43, 45, 41, 53, 48, 47, 47, 42, 43, 45, 40, 37, 48, 58, 58, 45, 43, 43, 40, 43, 55, 61, 44, 43, 37, 48, 54, 48, 56, 41, 44, 43, 61, 50, 56, 45, 43, 48, 52, 43, 56, 45, 40, 50, 40, 45, 52, 42, 56, 40, 42, 56, 45, 43, 42, 40, 56, 37, 48, 50, 56, 37, 58, 40, 37, 45, 40, 47, 45, 54, 40, 56, 37, 47, 55, 44, 53, 54, 58, 42, 37, 58, 53, 58, 49, 43, 54, 50, 37, 37, 40, 45, 37, 48, 59, 37, 43, 58, 37, 55, 43, 37, 40, 37, 58, 45, 37, 52, 43, 42, 40, 61, 40, 58, 41, 37, 45, 37, 50, 47, 48, 47, 47, 50, 54, 37, 50, 43, 47, 40, 61, 43, 45, 45, 58, 45, 37, 41, 43, 40, 37, 50, 55, 40, 37, 37, 52, 41, 42, 45, 40, 53, 48, 47, 53, 61, 43, 40, 42, 40, 37, 54, 37, 56, 58, 45, 43, 

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

In [ ]:
# Check that we have a GPU
!nvidia-smi
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun Apr 23 20:29:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.05    Driver Version: 525.85.05    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:82:00.0 Off |                  Off |
|  0

True

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=max_length, padding=True)
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset_val = dataset_val.map(preprocess_function, batched=True)

Map:   0%|          | 0/439493 [00:00<?, ? examples/s]

Map:   0%|          | 0/54378 [00:00<?, ? examples/s]

In [ ]:
from transformers import BertConfig
from transformers import BertForSequenceClassification

config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    max_position_embeddings=max_length+2,
    hidden_size=128,
    num_hidden_layers=2,
    num_attention_heads=8,
    type_vocab_size=1,
    num_labels=n_families_of_interest+1
)

model = BertForSequenceClassification(config=config)

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(261, 128, padding_idx=0)
      (position_embeddings): Embedding(514, 128)
      (token_type_embeddings): Embedding(1, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, ele

In [ ]:
model.num_parameters() / 1024 / 1024

1.8669214248657227

In [ ]:
from datasets import load_dataset, load_metric

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [27]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=save_folder,
    overwrite_output_dir=True,
    learning_rate=5e-4,
    num_train_epochs=15,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    logging_dir=os.path.join(save_folder, "logs"),            # directory for storing logs
    logging_steps=50,
    save_total_limit=2,
    prediction_loss_only=False,
    logging_strategy="epoch",
    bf16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,4.624100,4.499508,0.072474,0.016205
2,4.458200,4.376648,0.072511,0.017069
3,4.356000,4.280637,0.089135,0.023998
4,4.290300,4.231867,0.097668,0.029265
5,4.254500,4.213886,0.099176,0.030066


TrainOutput(global_step=17170, training_loss=4.396602108510484, metrics={'train_runtime': 2224.3062, 'train_samples_per_second': 987.933, 'train_steps_per_second': 7.719, 'total_flos': 1.2416350270311936e+16, 'train_loss': 4.396602108510484, 'epoch': 5.0})

In [33]:
training_args = TrainingArguments(
    output_dir=save_folder,
    overwrite_output_dir=True,
    learning_rate=5e-4,
    num_train_epochs=5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    logging_dir=os.path.join(save_folder, "logs"),            # directory for storing logs
    logging_steps=50,
    save_total_limit=2,
    prediction_loss_only=False,
    logging_strategy="epoch",
    bf16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.303800,0.859529,0.789143,0.780893
2,0.911600,0.582967,0.861690,0.858216
3,0.655700,0.422161,0.900474,0.898664
4,0.494400,0.323402,0.923572,0.922270
5,0.396600,0.273981,0.936886,0.935994


TrainOutput(global_step=17170, training_loss=0.7524396999672394, metrics={'train_runtime': 2163.4732, 'train_samples_per_second': 1015.712, 'train_steps_per_second': 7.936, 'total_flos': 1.2416350270311936e+16, 'train_loss': 0.7524396999672394, 'epoch': 5.0})

In [34]:
training_args = TrainingArguments(
    output_dir=save_folder,
    overwrite_output_dir=True,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    logging_dir=os.path.join(save_folder, "logs"),            # directory for storing logs
    logging_steps=50,
    save_total_limit=2,
    prediction_loss_only=False,
    logging_strategy="epoch",
    bf16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.368800,0.257706,0.939976,0.939266
2,0.320600,0.240826,0.944444,0.943806
3,0.284300,0.226632,0.947166,0.946612


TrainOutput(global_step=10302, training_loss=0.3245752801526706, metrics={'train_runtime': 1296.2348, 'train_samples_per_second': 1017.161, 'train_steps_per_second': 7.948, 'total_flos': 7448442657659136.0, 'train_loss': 0.3245752801526706, 'epoch': 3.0})

In [53]:
model.push_to_hub("PROTBERT")
tokenizer.push_to_hub("PROTBERT")

CommitInfo(commit_url='https://huggingface.co/mlewand/PROTBERT/commit/5b8473c5e4e3ed0ca3528fe2d011d8b9710f5d84', commit_message='Upload tokenizer', commit_description='', oid='5b8473c5e4e3ed0ca3528fe2d011d8b9710f5d84', pr_url=None, pr_revision=None, pr_num=None)

In [54]:
tokenizer.push_to_hub("PROTBERT")

CommitInfo(commit_url='https://huggingface.co/mlewand/PROTBERT/commit/277712a4caacf06153faebfeaac18f80b39785f3', commit_message='Upload tokenizer', commit_description='', oid='277712a4caacf06153faebfeaac18f80b39785f3', pr_url=None, pr_revision=None, pr_num=None)

In [47]:
datapoint = train_sentences.iloc[0]
encoded = tokenizer.encode(datapoint) 
print(f"datapoint : {datapoint}, len({len(datapoint)})")

datapoint : RIGIMTSGGDAPGMNLAIRAVARKALSSGLEAYGINYGFAGLVAGDIHEFKAADLDDMVSQGGTMLYSARYPEFAQEESQLKGIEQLKKFGIDALVVIGGDGSYHGALRLTEHGYNTIGLPGTIDNDIPFTDFTIGFDTALNTAVDAIDKIRDTAKSHQRVFAVQVMGRNAADIALWAGVASGADAVIAPGFDYDVEAIANKLKKNRANGKDYGIIVIAEGDANSDAAPEFIDQLKQYGDFDARATVIGHVQRGGVPSAKDRVLASKMGAYAVELL, len(276)


In [52]:
encoded_dataset[0]

{'sentence': 'RIGIMTSGGDAPGMNLAIRAVARKALSSGLEAYGINYGFAGLVAGDIHEFKAADLDDMVSQGGTMLYSARYPEFAQEESQLKGIEQLKKFGIDALVVIGGDGSYHGALRLTEHGYNTIGLPGTIDNDIPFTDFTIGFDTALNTAVDAIDKIRDTAKSHQRVFAVQVMGRNAADIALWAGVASGADAVIAPGFDYDVEAIANKLKKNRANGKDYGIIVIAEGDANSDAAPEFIDQLKQYGDFDARATVIGHVQRGGVPSAKDRVLASKMGAYAVELL',
 'labels': 233,
 'input_ids': [0,
  54,
  45,
  43,
  45,
  49,
  56,
  55,
  43,
  43,
  40,
  37,
  52,
  43,
  49,
  50,
  48,
  37,
  45,
  54,
  37,
  58,
  37,
  54,
  47,
  37,
  48,
  55,
  55,
  43,
  48,
  41,
  37,
  61,
  43,
  45,
  50,
  61,
  43,
  42,
  37,
  43,
  48,
  58,
  37,
  43,
  40,
  45,
  44,
  41,
  42,
  47,
  37,
  37,
  40,
  48,
  40,
  40,
  49,
  58,
  55,
  53,
  43,
  43,
  56,
  49,
  48,
  61,
  55,
  37,
  54,
  61,
  52,
  41,
  42,
  37,
  53,
  41,
  41,
  55,
  53,
  48,
  47,
  43,
  45,
  41,
  53,
  48,
  47,
  47,
  42,
  43,
  45,
  40,
  37,
  48,
  58,
  58,
  45,
  43,
  43,
  40,
  43,
  55,
  61,
  44,
  43,
  37,
  48,
  54,
  48,
  56,
  41,
 